# Metrics sensitivity to time horizon

The purpose of this notebook is to assess the sensitivity of climate metrics for a range of time horizons (e.g. $[20-100]$ years).

In [2]:
# Import librairies
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Climate functions
from aerometrics.utils.functions import emission_profile_function
from aerometrics.metrics.metrics import co2_ipcc_pulse_absolute_metrics, absolute_metrics, relative_metrics
from aerometrics.climate_models.co2_ipcc_climate_model import CO2_IPCC_ClimateModel
from aerometrics.climate_models.gwpstar_climate_model import GWPStarClimateModel
from aerometrics.climate_models.lwe_climate_model import LWEClimateModel
from aerometrics.climate_models.fair_climate_model import FairClimateModel

## Parameters

In [11]:
# Set parameters
climate_model = "FaIR" # "GWP*", "LWE, "FaIR"
co2_method = 'FaIR' # IPCC or FaIR
profile = "pulse"

if climate_model == "FaIR":
    start_year = 1765
else:
    start_year = 2000
t0 = 2020

model_settings = {"rcp": "RCP45"} if climate_model == "FaIR" else {"tcre": 0.00045}

settings = "Lee" # "Lee", "Bickel"
species_settings = {
    "CO2": {"ratio_erf_rf": 1.0, "efficacy_erf": 1.0},
    "Contrails": {"sensitivity_rf": 2.23e-12, "ratio_erf_rf": 0.42 if settings == "Lee" else 0.55, "efficacy_erf": 1.0 if settings == "Lee" else 0.38},
    "NOx - ST O3 increase": {"sensitivity_rf": 25.1e-12 * (14/46), "ratio_erf_rf": 1.37, "efficacy_erf": 1.0},
    "NOx - CH4 decrease and induced": {"ch4_loss_per_nox": -3.90, "ratio_erf_rf": 1.18, "efficacy_erf": 1.0},
    "H2O": {"sensitivity_rf": 0.0052e-12, "ratio_erf_rf": 1.0, "efficacy_erf": 1.0},
    "Soot": {"sensitivity_rf": 100.7e-12, "ratio_erf_rf": 1.0, "efficacy_erf": 1.0},
    "Sulfur": {"sensitivity_rf": -19.9e-12, "ratio_erf_rf": 1.0, "efficacy_erf": 1.0},
}

unit_values = {
    "CO2": 1*10**10,
    "Contrails": 1*10**10,
    "NOx - ST O3 increase": 1*10**10,
    "NOx - CH4 decrease and induced": 1*10**10,
    "H2O": 1*10**12,
    "Soot": 1*10**14,
    "Sulfur": 1*10**10
}

## Metrics calculation

In [13]:
# Initialisation
time_horizon_range = range(15,110,5)
gwp_rf = np.zeros((len(time_horizon_range),6))
gwp_erf = np.zeros((len(time_horizon_range),6))
egwp_rf = np.zeros((len(time_horizon_range),6))
egwp_erf = np.zeros((len(time_horizon_range),6))
gtp = np.zeros((len(time_horizon_range),6))
ratr = np.zeros((len(time_horizon_range),6))

# Loop through species and time horizons
for j, species in enumerate(["Contrails", "NOx - ST O3 increase", "NOx - CH4 decrease and induced", "H2O", "Soot", "Sulfur"]):

    for k, time_horizon in enumerate(time_horizon_range):
        end_year = t0 + time_horizon        
        emission_profile_co2 = emission_profile_function(start_year,t0,time_horizon, profile, unit_values["CO2"])
        emission_profile = emission_profile_function(start_year,t0,time_horizon, profile, unit_values[species])

        # CO2 absolute metrics
        if co2_method == 'IPCC':
            agwp_rf_co2, agwp_erf_co2, aegwp_rf_co2, aegwp_erf_co2, agtp_co2, iagtp_co2, atr_co2 = co2_ipcc_pulse_absolute_metrics(time_horizon)
        elif co2_method == 'FaIR':
            results_co2 = FairClimateModel(
                start_year,
                end_year,
                "CO2",
                emission_profile_co2,
                species_settings["CO2"],
                model_settings
            ).run()
            radiative_forcing_co2_unit = results_co2["radiative_forcing"] / unit_values["CO2"]
            effective_radiative_forcing_co2_unit = results_co2["effective_radiative_forcing"] / unit_values["CO2"]
            temperature_co2_unit = results_co2["temperature"] / unit_values["CO2"]
            agwp_rf_co2, agwp_erf_co2, aegwp_rf_co2, aegwp_erf_co2, agtp_co2, iagtp_co2, atr_co2 = absolute_metrics(
                radiative_forcing_co2_unit, 
                effective_radiative_forcing_co2_unit, 
                species_settings["CO2"].get("efficacy_erf", 1.0), 
                temperature_co2_unit, 
                time_horizon
            )

        # Species absolute metrics
        if climate_model == "GWP*":
            results = GWPStarClimateModel(start_year, end_year, species, emission_profile, species_settings[species], model_settings).run()
        elif climate_model == "LWE":
            sresults = LWEClimateModel(start_year, end_year, species, emission_profile, species_settings[species], model_settings).run()
        elif climate_model == "FaIR":
            results = FairClimateModel(start_year, end_year, species, emission_profile, species_settings[species], model_settings).run()

        unit_value = unit_values[species]
        radiative_forcing_unit = results["radiative_forcing"] / unit_value
        effective_radiative_forcing_unit = results["effective_radiative_forcing"] / unit_value
        temperature_unit = results["temperature"] / unit_value
        agwp_rf, agwp_erf, aegwp_rf, aegwp_erf, agtp, iagtp, atr = absolute_metrics(
            radiative_forcing_unit, 
            effective_radiative_forcing_unit, 
            species_settings[species].get("efficacy_erf", 1.0),
            temperature_unit,
            time_horizon
        )

        # Relative metrics
        gwp_rf[k,j], gwp_erf[k,j], egwp_rf[k,j], egwp_erf[k,j], gtp[k,j], igtp, ratr[k,j] = relative_metrics(
            agwp_rf_co2, 
            agwp_erf_co2, 
            aegwp_rf_co2, 
            aegwp_erf_co2, 
            agtp_co2,
            iagtp_co2, 
            atr_co2, 
            agwp_rf, 
            agwp_erf,
            aegwp_rf,
            aegwp_erf, 
            agtp, 
            iagtp, 
            atr
        )

In [14]:
# Plot results
fig, axes = plt.subplots(3, 2, figsize=(12, 15))

# Ratio based on 2018 data
ratio_contrails = 61.3/1033.7 # Million km / MtCO2
ratio_nox = 1.4*46/14/1033.7 # MtNOx / MtCO2
ratio_h2o = 382.55/1033.7 # MtH2O / MtCO2
ratio_soot = 0.0093/1033.7 # MtBC / MtCO2
ratio_sulfur = 0.37/1033.7 # MtSOx / MtCO2

for i, ax in enumerate(axes.flat):
    if i == 0:
        x_axis = time_horizon_range
        ax.plot(x_axis, ratio_contrails*gwp_rf[:, i+0]+ratio_nox*(gwp_rf[:, i+2]+gwp_rf[:, i+1])+ratio_h2o*gwp_rf[:, i+3]+ratio_soot*gwp_rf[:, i+4]+ratio_sulfur*gwp_rf[:, i+5], label=r'$GWP_{RF}$', color='C0')
        ax.plot(x_axis, ratio_contrails*gwp_erf[:, i+0]+ratio_nox*(gwp_erf[:, i+2]+gwp_erf[:, i+1])+ratio_h2o*gwp_erf[:, i+3]+ratio_soot*gwp_erf[:, i+4]+ratio_sulfur*gwp_erf[:, i+5], label=r'$GWP_{ERF}$', color='C1')
        ax.plot(x_axis, ratio_contrails*gtp[:, i+0]+ratio_nox*(gtp[:, i+2]+gtp[:, i+1])+ratio_h2o*gtp[:, i+3]+ratio_soot*gtp[:, i+4]+ratio_sulfur*gtp[:, i+5], label=r'$GTP$', color='C2')
        ax.plot(x_axis, ratio_contrails*ratr[:, i+0]+ratio_nox*(ratr[:, i+2]+ratr[:, i+1])+ratio_h2o*ratr[:, i+3]+ratio_soot*ratr[:, i+4]+ratio_sulfur*ratr[:, i+5], label=r'$r\text{-}ATR$', color='C3')
        ax.plot(20, 3, marker="o", color="C1", markersize=8, label=r'$GWP_{ERF}$ (Lee et al.)', linestyle="None")
        ax.plot(50, 1.3, marker="o", color="C1", markersize=8)
        ax.plot(100, 0.7, marker="o", color="C1", markersize=8)
        ax.plot(20, 0.3, marker="o", color="C2", markersize=8, label=r'$GTP$ (Lee et al.)', linestyle="None")
        ax.plot(50, 0.0, marker="o", color="C2", markersize=8)
        ax.plot(100, 0.1, marker="o", color="C2", markersize=8)
        ax.vlines(x=10, ymin=0, ymax=1, color="C0", label=r'$GWP_{RF}$ (IPCC AR5)', linestyle="--", alpha=0.7)
        ax.vlines(x=110, ymin=0, ymax=1, color="C2", label=r'$GTP$ (IPCC AR5)', linestyle="--", alpha=0.7)
        ax.legend(ncol=2)
        ax.set_ylabel(r'Aggregated metrics for 2018 [$kgCO_2\text{-}eq/kgCO_2$]')
    elif i == 1:
        ax.plot(x_axis, gwp_rf[:, i-1], label=r'$GWP_{RF}$', color='C0')
        ax.plot(x_axis, gwp_erf[:, i-1], label=r'$GWP_{ERF}$', color='C1')
        ax.plot(x_axis, gtp[:, i-1], label=r'$GTP$', color='C2')
        ax.plot(x_axis, ratr[:, i-1], label=r'$r\text{-}ATR$', color='C3')
        ax.plot(20, 39, marker="o", color="C1", markersize=8, label=r'$GWP_{ERF}$ (Lee et al.)')
        ax.plot(50, 18, marker="o", color="C1", markersize=8)
        ax.plot(100, 11, marker="o", color="C1", markersize=8)
        ax.plot(20, 11, marker="o", color="C2", markersize=8, label=r'$GTP$ (Lee et al.)')
        ax.plot(50, 1.8, marker="o", color="C2", markersize=8)
        ax.plot(100, 1.5, marker="o", color="C2", markersize=8)
        ax.set_ylabel(r'Metrics [$kgCO_2\text{-}eq/km_{\text{flights}}$]')
    elif i == 2:
        ax.plot(x_axis, gwp_rf[:, i-1]+gwp_rf[:, i], color='C0')
        ax.plot(x_axis, gwp_erf[:, i-1]+gwp_erf[:, i], color='C1')
        ax.plot(x_axis, gtp[:, i-1]+gtp[:, i], color='C2')
        ax.plot(x_axis, ratr[:, i-1]+ratr[:, i], color='C3')
        ax.plot(20, 619, marker="o", color="C1", markersize=8, label=r'$GWP_{ERF}$ (Lee et al.)')
        ax.plot(50, 205, marker="o", color="C1", markersize=8)
        ax.plot(100, 114, marker="o", color="C1", markersize=8)
        ax.plot(20, -222, marker="o", color="C2", markersize=8, label=r'$GTP$ (Lee et al.)')
        ax.plot(50, -69, marker="o", color="C2", markersize=8)
        ax.plot(100, 13, marker="o", color="C2", markersize=8)
        ax.vlines(x=20, ymin=92, ymax=415, color="C0", label=r'$GWP_{RF}$ (IPCC AR5)', linestyle="--", alpha=0.7)
        ax.hlines(xmin=18, xmax=22, y=92, color="C0", alpha=0.7)
        ax.hlines(xmin=18, xmax=22, y=415, color="C0", alpha=0.7)
        ax.vlines(x=100, ymin=-21, ymax=75, color="C0", linestyle="--", alpha=0.7)
        ax.hlines(xmin=98, xmax=102, y=-21, color="C0", alpha=0.7)
        ax.hlines(xmin=98, xmax=102, y=75, color="C0", alpha=0.7)
        ax.vlines(x=20, ymin=-590, ymax=-121, color="C2", label=r'$GTP$ (IPCC AR5)', linestyle="--", alpha=0.7)
        ax.hlines(xmin=18, xmax=22, y=-121, color="C2", alpha=0.7)
        ax.hlines(xmin=18, xmax=22, y=-590, color="C2", alpha=0.7)
        ax.vlines(x=100, ymin=-9.5, ymax=8.6, color="C2", linestyle="--", alpha=0.7)
        ax.hlines(xmin=98, xmax=102, y=-9.5, color="C2", alpha=0.7)
        ax.hlines(xmin=98, xmax=102, y=8.6, color="C2", alpha=0.7)
        ax.set_ylabel(r'Metrics [$kgCO_2\text{-}eq/kgNO_x$]')
        ax.set_yscale('symlog', linthresh=10)

    else:
        x_axis = time_horizon_range
        ax.plot(x_axis, gwp_rf[:, i], label=r'$GWP_{RF}$', color='C0')
        ax.plot(x_axis, gwp_erf[:, i], label=r'$GWP_{ERF}$', color='C1')
        ax.plot(x_axis, gtp[:, i], label=r'$GTP$', color='C2')
        ax.plot(x_axis, ratr[:, i], label=r'$r\text{-}ATR$', color='C3')
        if i==3:
            ax.plot(20, 0.22, marker="o", color="C1", markersize=8, label=r'$GWP_{ERF}$ (Lee et al.)')
            ax.plot(50, 0.1, marker="o", color="C1", markersize=8)
            ax.plot(100, 0.06, marker="o", color="C1", markersize=8)
            ax.plot(20, 0.07, marker="o", color="C2", markersize=8, label=r'$GTP$ (Lee et al.)')
            ax.plot(50, 0.01, marker="o", color="C2", markersize=8)
            ax.plot(100, 0.008, marker="o", color="C2", markersize=8)
            ax.set_ylabel(r'Metrics [$kgCO_2\text{-}eq/kgH_2O$]')
        elif i==4:
            ax.plot(20, 4288, marker="o", color="C1", markersize=8, label=r'$GWP_{ERF}$ (Lee et al.)')
            ax.plot(50, 2018, marker="o", color="C1", markersize=8)
            ax.plot(100, 1166, marker="o", color="C1", markersize=8)
            ax.plot(20, 1245, marker="o", color="C2", markersize=8, label=r'$GTP$ (Lee et al.)')
            ax.plot(50, 195, marker="o", color="C2", markersize=8)
            ax.plot(100, 161, marker="o", color="C2", markersize=8)
            ax.set_ylabel(r'Metrics [$kgCO_2\text{-}eq/kgBC$]')
        else:
            ax.plot(20, -832, marker="o", color="C1", markersize=8, label=r'$GWP_{ERF}$ (Lee et al.)')
            ax.plot(50, -392, marker="o", color="C1", markersize=8)
            ax.plot(100, -226, marker="o", color="C1", markersize=8)
            ax.plot(20, -241, marker="o", color="C2", markersize=8, label=r'$GTP$ (Lee et al.)')
            ax.plot(50, -38, marker="o", color="C2", markersize=8)
            ax.plot(100, -31, marker="o", color="C2", markersize=8)
            ax.set_ylabel(r'Metrics [$kgCO_2\text{-}eq/kgSO_2$]')
    ax.set_xlim(min(time_horizon_range),max(time_horizon_range))
    ax.set_xlabel('Time horizon')
    ax.grid(True)
    if i == 0:
        ax.set_title('(a) Total non-CO₂')
    elif i == 1:
        ax.set_title('(b) Contrails')
    elif i == 2:
        ax.set_title('(c) NOx')
    elif i == 3:
        ax.set_title('(d) Water vapour')
    elif i == 4:
        ax.set_title('(e) Soot')
    elif i == 5:
        ax.set_title('(f) Sulfur')

plt.tight_layout()
plt.savefig("figures/metric_calculation.pdf")
plt.show()